<a href="https://colab.research.google.com/github/CarlosARossi/dataScienceCoder/blob/main/Desafio_APIS_Carlos_Rossi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Descarga de datos desde APIs públicas


### Objetivos generales

1. Buscar información en APIs públicas (i.e Twitter, NewsAPI, Spotify, Google Apis, etc).
2. Extraer datos e importarlos a un dataframe realizando una exploración simple (i.e filas, columnas, tipos de datos). Se sugiere que estos datos complementen el dataset elegido en el Desafio “Elección de potenciales Datasets e importe con la librería Pandas”







## 1. Importación de librerias y datasets

 La elección de la API de Google Maps para adicionar información a las empresas en el dataset se basa en su popularidad, amplia base de datos geográfica y funcionalidades. Su API es fácil de usar, con documentación extensa y recursos disponibles. La elección se basa en su cobertura geográfica, funcionalidades completas y facilidad de integración.


In [1]:
!pip install googlemaps

import googlemaps

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40718 sha256=f16b0272f212383e358813f11a1dea2635da25226916d6381634bca21bdaed5a
  Stored in directory: /root/.cache/pip/wheels/17/f8/79/999d5d37118fd35d7219ef57933eb9d09886c4c4503a800f84
Successfully built googlemaps


In [2]:
# Configura tu clave de API de Google Places
API_KEY = 'AQUI SE ENCONTRABA LA KEY OBTENIDA EN GOOGLE CLOUD PLATFORM'

Importo el dataset generado en el notebook [ProyectoDS_ParteII_+Rossi.ipynb](https://github.com/CarlosARossi/dataScienceCoder/blob/main/ProyectoDS_ParteII_%2BRossi.ipynb), en el cual se aplicó una exploración de datos (EDA).

In [3]:
import pandas as pd

In [4]:
url = 'https://raw.githubusercontent.com/CarlosARossi/dataScienceCoder/main/Datasets/Salary_Data_2022_REV15_USD_Months.csv'
salaries = pd.read_csv(url)
salaries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31762 entries, 0 to 31761
Data columns (total 28 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   ID                                                               31762 non-null  int64  
 1   Timestamp                                                        31762 non-null  object 
 2   Age.Range                                                        31742 non-null  object 
 3   Years.of.Experience                                              31260 non-null  float64
 4   Industry                                                         31558 non-null  object 
 5   Job.Title                                                        31758 non-null  object 
 6   Company.Name                                                     31760 non-null  object 
 7   Education                               

## 2. Análisis de valores útiles para la API

Primero, se realizará un análisis para determinar la cantidad de compañías únicas y países únicos presentes en el dataset **salaries**.

In [5]:
# Contar la cantidad de instancias únicas de las caracteristicas considerads como variables nominales

print("Cantidad de Compañias:", len(salaries['Company.Name'].unique()))
print("Cantidad de Paises:", len(salaries['Country'].unique()))

Cantidad de Compañias: 14718
Cantidad de Paises: 55


Luego, se generará un nuevo dataset que contenga únicamente las compañías que aparezcan más de 50 veces en el dataset original. Además, se incluirá el país correspondiente a cada una de estas compañías.

In [6]:
# Cuenta el número de repeticiones de cada empresa
company_counts = salaries['Company.Name'].value_counts()

# Filtra las empresas con 50 o más apariciones
repeated_companies = company_counts[company_counts >= 50]

# Filtra el dataset original para mantener solo las filas correspondientes a las empresas repetidas
filtered_salaries = salaries[salaries['Company.Name'].isin(repeated_companies.index)]

# Obtiene el país correspondiente a cada empresa repetida
countries = filtered_salaries.groupby('Company.Name')['Country'].first().reset_index()

# Crea el nuevo dataset con las columnas "Company.Name" y "Country"
company_country = pd.DataFrame({'Company.Name': repeated_companies.index, 'Country': countries['Country']})

# Imprime el resultado
company_country.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Company.Name  48 non-null     object
 1   Country       48 non-null     object
dtypes: object(2)
memory usage: 896.0+ bytes


Por ultimo, se reemplazará el valor "US" por "UNITED STATES" en la columna del país. Esto es necesario para que la API de Google Maps pueda identificar correctamente el país durante el proceso de enriquecimiento con información.

In [7]:
# Reemplaza "US" por "United States" en la columna "Country"
company_country['Country'] = company_country['Country'].replace('US', 'UNITED STATES')

# Imprime el resultado
company_country

,Company.Name,Country
0,AMAZON,UNITED STATES
1,PUBLIC SCHOOL,UNITED STATES
2,DELOITTE,UNITED STATES
3,GOVERNMENT,UNITED STATES
4,EY,UNITED STATES
5,ACCENTURE,UNITED STATES
6,KPMG,UNITED STATES
7,PWC,UNITED STATES
8,JP MORGAN CHASE,UNITED STATES
9,SELF-EMPLOYED,UNITED STATES


## 3. Procesamiento de datos con API de Google Maps


En el siguiente código se utilizará la API de Google Maps para enriquecer la información de las empresas en elegidas en el punto anterior. Para esto, Se realizará una búsqueda utilizando el nombre de la compañia y el país, obteniendo para cada una detalles adicionales, como dirección, URL, sitio web, puntuación en Google Maps y coordenadas. La información se agrega en un nuevo DataFrame, permitiendo complementar el dataset original con datos más detallados de las empresas.

In [8]:
# Crea una instancia del cliente de Google Maps
gmaps = googlemaps.Client(key=API_KEY)

# Función para obtener información adicional de una empresa en Google Maps
def gmaps_information(company, country):

    # Realiza la solicitud de búsqueda
    results = gmaps.places_autocomplete(f'{company}, {country}')

     # Verifica si se encontraron resultados y si la solicitud fue exitosa
    if len(results) > 0:
        place_id = results[0]['place_id']

        # Realiza la solicitud de detalles del lugar utilizando el Place ID
        detalles = gmaps.place(place_id)

        # Verifica si se encontraron detalles y si la solicitud fue exitosa
        if 'result' in detalles and detalles['status'] == 'OK':
            primer_resultado = detalles['result']

            # Obtén la información adicional de la empresa
            address = primer_resultado.get('formatted_address', 'Dirección no disponible')
            url = primer_resultado.get('url', 'URL no disponible')
            website = primer_resultado.get('website', 'Website no disponible')
            score = primer_resultado.get('rating', 'Puntuación no disponible')
            coordinates = primer_resultado['geometry']['location']

            # Devuelve la información obtenida
            return {'Address': address,
                    'URL': url,
                    'Website': website,
                    'Score': score,
                    'Coordinates': coordinates}
    else:
        # No se encontraron resultados
        return {'Error': 'No se encontraron resultados para la empresa'}

# Crear una lista para almacenar los resultados
company_results = []

# Iterar sobre cada fila del dataset
for _, row in company_country.iterrows():
    company = row['Company.Name']
    country = row['Country']

    # Obtener información adicional de la empresa
    company_information = gmaps_information(company, country)

    # Agregar los resultados a la lista
    company_results.append({
        'Company.Name': company,
        'Country': country,
        'Direction': company_information.get('Address', ''),
        'URL': company_information.get('URL', ''),
        'Website': company_information.get('Website', ''),
        'Score.Google.Maps': company_information.get('Score', ''),
        'Coordinates': company_information.get('Coordinates', {})
    })

# Crear un nuevo dataframe con los resultados
company_gmaps = pd.DataFrame(company_results)

company_gmaps

,Company.Name,Country,Direction,URL,Website,Score.Google.Maps,Coordinates
0,AMAZON,UNITED STATES,"400 9th Ave N, Seattle, WA 98109, USA",https://maps.google.com/?cid=10321563756011321016,http://www.amazon.com/,3.7,"{'lat': 47.62245530000001, 'lng': -122.3394743}"
1,PUBLIC SCHOOL,UNITED STATES,"4001 S Ocean Dr, Hollywood, FL 33019, USA",https://maps.google.com/?cid=12001204246517214969,Website no disponible,Puntuación no disponible,"{'lat': 25.9874119, 'lng': -80.1190258}"
2,DELOITTE,UNITED STATES,"111 S Wacker Dr STE 1800, Chicago, IL 60606, USA",https://maps.google.com/?cid=6706465131668850308,https://www2.deloitte.com/us/en/footerlinks/of...,4.6,"{'lat': 41.8804457, 'lng': -87.63641609999999}"
3,GOVERNMENT,UNITED STATES,"Government, NE, USA","https://maps.google.com/?q=Government,+NE,+USA...",Website no disponible,Puntuación no disponible,"{'lat': 40.57391459999999, 'lng': -101.0033112}"
4,EY,UNITED STATES,"2323 Victory Ave Suite 2000, Dallas, TX 75202,...",https://maps.google.com/?cid=3460885643605346203,http://www.ey.com/,4.7,"{'lat': 32.7858231, 'lng': -96.8104446}"
5,ACCENTURE,UNITED STATES,"5205 N O'Connor Blvd #1400, Irving, TX 75039, USA",https://maps.google.com/?cid=10195265057843093163,https://www.accenture.com/us-en/about/company/...,4.6,"{'lat': 32.8703634, 'lng': -96.9395894}"
6,KPMG,UNITED STATES,"Aon Center, 200 E Randolph St #5500, Chicago, ...",https://maps.google.com/?cid=16914892130222467968,http://www.kpmg.com/,4.3,"{'lat': 41.8852828, 'lng': -87.62151209999999}"
7,PWC,UNITED STATES,"101 Seaport Blvd, Boston, MA 02210, USA",https://maps.google.com/?cid=13309566566665129107,Website no disponible,3.8,"{'lat': 42.3514278, 'lng': -71.0452535}"
8,JP MORGAN CHASE,UNITED STATES,"245 Park Ave, New York, NY 10029, USA",https://maps.google.com/?cid=1930836371954593676,https://www.chase.com/,2.4,"{'lat': 40.75460949999999, 'lng': -73.9746638}"
9,SELF-EMPLOYED,UNITED STATES,"50 Uptown Grayton Cir UNIT 2, Santa Rosa Beach...",https://maps.google.com/?cid=9705276742990708833,Website no disponible,5,"{'lat': 30.336034, 'lng': -86.163208}"


Por ultimo, se utilizará la biblioteca folium para generar un mapa interactivo que muestra la ubicación de las empresas en el dataset.

Se utilizará las coordenadas de cada empresa para colocar un marcador en el mapa. Además, se personalizará el marcador con información adicional de la empresa, como su nombre, dirección, URL y puntuación en Google Maps.

Esto permitirá visualizar de manera gráfica la distribución geográfica de las empresas y acceder a detalles relevantes al hacer clic en los marcadores.

In [9]:
import folium

# Crear un mapa centrado en una ubicación inicial
mapa = folium.Map(location=[0, 0], zoom_start=2, width=1500, height=700)

# Iterar sobre los datos del dataset
for _, row in company_gmaps.iterrows():
    latitud = row['Coordinates']['lat']
    longitud = row['Coordinates']['lng']
    nombre_empresa = row['Company.Name']
    direccion = row['Direction']
    website = row['Website']
    puntuacion = row['Score.Google.Maps']

    # Crear el texto emergente con los datos adicionales
    popup_text = f"<strong>{nombre_empresa}</strong><br>" \
                 f"<strong>Dirección:</strong> {direccion}<br>" \
                 f"<strong>Website:</strong> {website}<br>" \
                 f"<strong>Puntuación en Google Maps:</strong> {puntuacion}/5"

    # Agregar un marcador en cada ubicación
    folium.Marker(
        location=[latitud, longitud],
        popup=popup_text,
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(mapa)

# Mostrar el mapa
mapa